--Google Colab--

In [ ]:
!pip install evaluate

--Contrato legal--

Objetivo: detectar si un contrato contiene una cláusula de terminación anticipada.

Dataset simulado: documentos largos tipo contratos (puedes generar texto largo o usar un dataset legal).

Modelo: allenai/longformer-base-4096

Tokenización: se mantiene secuencia completa hasta 4.096 tokens.

Tarea: clasificación binaria → “contiene cláusula de terminación anticipada” sí/no.

Ventaja: el modelo ve todo el contrato de una vez, sin cortar cláusulas, y la atención global permite que referencias dispersas se unan.

--Vinculación con Drive--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

--Definiendo la ruta del directorio donde se encontraran el modelo entrenado y resultados--

In [ ]:
import os
drive_dir = "/content/drive/MyDrive/Modelos_entrenados_ML_DL/Longformer_Contrato_Legal"
os.makedirs(drive_dir, exist_ok=True)

--Importación de librerias necesarias--

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
import torch
from datasets import Dataset, load_dataset
import numpy as np
import pandas as pd
import random
import evaluate
import json
from torch.nn import CrossEntropyLoss

--Dataset CUAD filtrado con palabras clave--

In [ ]:
!git clone  https://github.com/TheAtticusProject/cuad.git
!unzip cuad/data.zip

In [ ]:
keywords = ["termination", "terminate", "end of contract", "contract end", "cancellation", "cancel"]

with open("/content/CUADv1.json", "r", encoding="utf-8") as f:
    cuad_json = json.load(f)

examples = []
for contract in cuad_json["data"]:
    for para in contract["paragraphs"]:
        context = para["context"]
        for qa in para["qas"]:
            if "termination" in qa["question"].lower():
                examples.append({
                    "text": context,
                    "label": 0 if qa["is_impossible"] else 1
                })

dataset = Dataset.from_list(examples)


--Dividir el Dataset en entrenamiento y validación--

In [ ]:
split = dataset.train_test_split(test_size=0.2, seed=12)
train_dataset = split['train']
val_dataset = split['test']

--Tokenización--

In [ ]:
# Cargar el tokenizador Longformer
model_name = "allenai/longformer-base-4096"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenización
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",     # Relleno hasta la longitud máxima
        truncation=True,          # Truncar si excede la longitud máxima
        max_length=2048,          # Longitud máxima para Longformer
    )

# Tokenizar los conjuntos de datos
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

--Configuración del modelo--

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,     # Modelo Longformer preentrenado
    num_labels=2    # Número de clases (presente o ausente de cláusula de terminación anticipada)
)

--Calculo de pesos según balance de classes--

In [ ]:
labels = np.array([ex["label"] for ex in examples])
class_counts = np.bincount(labels)
class_weights = torch.tensor([1.0, class_counts[0] / class_counts[1]]).to(model.device)

def weighted_loss(logits, labels):
    loss_fct = CrossEntropyLoss(weight=class_weights)
    return loss_fct(logits, labels)

--Metricas de evaluación--

In [ ]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels, average='macro')
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}

--Configuración del entrenamiento--

In [ ]:
training_args = TrainingArguments(
    output_dir=f"{drive_dir}/results",       # Directorio para guardar los resultados
    num_train_epochs=3,                      # Número de épocas de entrenamiento
    per_device_train_batch_size=2,           # Tamaño del batch de entrenamiento
    per_device_eval_batch_size=2,            # Tamaño del batch de evaluación
    learning_rate=2e-5,                      # Tasa de aprendizaje
    eval_strategy="epoch",             # Evaluar al final de cada época
    save_strategy="epoch",                   # Guardar el modelo al final de cada época
    load_best_model_at_end=True,             # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",              # Métrica para determinar el mejor modelo
    greater_is_better=True,                  # Indica si una métrica mayor es mejor
    logging_dir=f"{drive_dir}/logs",                    # Directorio para guardar los logs
    seed=12,                                 # Semilla para reproducibilidad
    fp16=True,                               # Usar precisión mixta (FP16) si es compatible
    gradient_accumulation_steps=4,           # Acumulación de gradientes para simular un batch más grande
    report_to="none"
)

--Trainer--

In [ ]:
trainer = Trainer(
    model=model,                             # El modelo a entrenar
    args=training_args,                      # Los argumentos de entrenamiento
    train_dataset=train_dataset,             # El conjunto de datos de entrenamiento
    eval_dataset=val_dataset,                # El conjunto de datos de evaluación
    tokenizer=tokenizer,                     # El tokenizador
    compute_metrics=compute_metrics,         # La función para calcular métricas
)

--Entrenamiento--

In [ ]:
trainer.train()

--Guardado del modelo entrenado--

In [ ]:
trainer.save_model(f"{drive_dir}/longformer-modelo-contratos")

--Inferencia con sliding window (aplicas el modelo ya entrenado a nuevos contratos que no ha visto antes, para predecir si contienen la cláusula.)--

In [ ]:
def classify_contract(contract_text, classifier, tokenizer, max_length=4096, stride=512):
    inputs = tokenizer(
        contract_text,
        max_length=max_length,
        stride=stride,
        truncation=True,
        return_overflowing_tokens=True,
        padding="max_length",
        return_tensors="pt"
    )
    probs = []
    for i in range(inputs.input_ids.shape[0]):
        chunk_text = tokenizer.decode(inputs.input_ids[i], skip_special_tokens=True)
        result = classifier(chunk_text)
        probs.append(result[0]['score'] if result[0]['label'] == 'LABEL_1' else 1 - result[0]['score'])
    final_score = max(probs)
    return "sí" if final_score > 0.5 else "no"

--Pipeline para inferencia--

In [ ]:
classifier = pipeline(
    "text-classification",
    model=f"{drive_dir}/longformer-modelo-contratos",
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

--Pruebas con nuevos contratos--

In [ ]:
# Contratos de prueba
new_contracts = [
    "El presente contrato podrá ser rescindido en cualquier momento si ambas partes así lo acuerdan.",
    "Este contrato de compraventa se mantendrá vigente por 5 años sin posibilidad de terminación anticipada."
]

for c in new_contracts:
    pred = classify_contract(c, classifier, tokenizer)
    print(f"Contrato: {c[:60]}... \nPredicción: {pred}\n")